In [6]:
import os
import chromadb
from tqdm import tqdm
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

In [7]:
llm = ChatOllama(model="llama2:7b")
emb = OllamaEmbeddings(model="llama2:7b")

In [8]:
llm.invoke("Hello world!")

AIMessage(content="\nHello there! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'model': 'llama2:7b', 'created_at': '2024-04-10T09:30:17.182952Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 2190328083, 'load_duration': 1943125, 'prompt_eval_duration': 225608000, 'eval_count': 28, 'eval_duration': 1961974000}, id='run-b445cf2d-3c04-4d9a-b59b-2c74510897c0-0')

In [9]:
class OllamaEmbeddingFn(chromadb.EmbeddingFunction):
    def __call__(self, input: chromadb.Documents) -> chromadb.Embeddings:
        return emb.embed_documents(input)

In [10]:
# initialise chroma collection
# adapted from https://docs.trychroma.com/usage-guide#using-collections
client = chromadb.PersistentClient(path="documents/chroma_db")
collection = client.get_or_create_collection(
    name="helpsheets", embedding_function=OllamaEmbeddingFn()
)

In [11]:
# paths of documents to index
helpsheet_dir = r"documents/helpsheet collection"
helpsheet_paths = [
    f
    for f in os.listdir(helpsheet_dir)
    if os.path.isfile(os.path.join(helpsheet_dir, f))
]

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# index documents (takes forever)
# adapted from https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf/#using-pypdf
if collection.count() == 0:
    for i, hs_path in tqdm(enumerate(helpsheet_paths)):
        print(i, hs_path)
        # load document
        loader = PyPDFLoader(os.path.join(helpsheet_dir, hs_path))
        docs = loader.load()
        # split document into chunks
        splits = text_splitter.split_documents(docs)
        # unique ids for each chunk
        ids = [f"{i} - {j}" for j in range(len(splits))]
        # add chunks into chroma collection
        collection.add(
            ids=ids,
            metadatas=[d.metadata for d in splits],
            documents=[d.page_content for d in splits],
        )

0it [00:00, ?it/s]Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)


0 9 - AVL Trees and Balancing.pdf


1it [00:25, 25.03s/it]Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)


1 10 - Graphs.pdf


2it [00:47, 23.75s/it]Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)


...

13it [08:23, 34.84s/it]Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)


13 1 - Complexities and Searching.pdf


14it [08:48, 31.68s/it]Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 67 0 (offset 0)
Ignoring wrong pointing object 74 0 (offset 0)
Ignoring wrong pointing object 77 0 (offset 0)


14 11 - Graph Operations and Analysis.pdf


15it [09:43, 38.89s/it]


In [14]:
# initialise langchain vector store from chroma client
# adapted from https://python.langchain.com/docs/integrations/vectorstores/chroma/#passing-a-chroma-client-into-langchain
vectorstore = Chroma(
    client=client,
    collection_name="helpsheets",
    embedding_function=emb,
)
retriever = vectorstore.as_retriever()

In [15]:
# code for chat history (including next 2 code blocks)
# adapted from https://python.langchain.com/docs/use_cases/question_answering/chat_history/
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [17]:
qa_system_prompt = """You are a computer science tutoring assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
Do not mention "the context". \
If you don't know the answer, just say that you don't know. \
Use five sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [18]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [19]:
# streams llm output
# adapted from https://python.langchain.com/docs/use_cases/question_answering/streaming/
def query_and_print(input: str, session_id="abc123"):
    for chunk in conversational_rag_chain.stream(
        {"input": input},
        config={
            "configurable": {"session_id": session_id}
        },  # constructs a key "abc123" in `store`.
    ):
        if "answer" in chunk:
            print(chunk["answer"], end="")
        else:
            print(chunk)

In [21]:
query_and_print("What did i ask you just now?")

{'input': 'What did i ask you just now?'}
{'chat_history': [HumanMessage(content='What is an MST?'), AIMessage(content="An MST (Minimum Spanning Tree) is a subgraph of a graph that connects all the vertices together while minimizing the total weight of the edges. In other words, it is a subset of the edges of the original graph that connect all the vertices together in a way that minimizes the sum of the weights of the edges.\n\nFor example, consider a graph with 5 vertices labeled A, B, C, D, and E, and 7 edges: AB, AC, AD, BE, BC, BD, and DE. The MST of this graph would be the subset of edges that connects all the vertices together in a way that minimizes the total weight of the edges. In this case, the MST would be {AB, AC, AD, BD}, which has a total weight of 4.\n\nMSTs have many applications in graph theory and computer science, such as:\n\n* Network flow optimization: MSTs can be used to optimize network flow problems by finding the minimum cost flow that satisfies all constraint

In [22]:
query_and_print("How to solve it using Prim's algorithm?")

{'input': "How to solve it using Prim's algorithm?"}
{'chat_history': [HumanMessage(content='What is an MST?'), AIMessage(content="An MST (Minimum Spanning Tree) is a subgraph of a graph that connects all the vertices together while minimizing the total weight of the edges. In other words, it is a subset of the edges of the original graph that connect all the vertices together in a way that minimizes the sum of the weights of the edges.\n\nFor example, consider a graph with 5 vertices labeled A, B, C, D, and E, and 7 edges: AB, AC, AD, BE, BC, BD, and DE. The MST of this graph would be the subset of edges that connects all the vertices together in a way that minimizes the total weight of the edges. In this case, the MST would be {AB, AC, AD, BD}, which has a total weight of 4.\n\nMSTs have many applications in graph theory and computer science, such as:\n\n* Network flow optimization: MSTs can be used to optimize network flow problems by finding the minimum cost flow that satisfies all

In [ ]:
store.clear()